In [45]:
import pandas as pd
import os
import random

# Data Preprocessing

In [46]:
data = pd.read_csv('kaggle_data/scopusBYUEngr17_21.csv')

In [47]:
data.columns

Index(['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'Issue', 'Art. No.', 'Page start', 'Page end', 'Page count', 'Cited by',
       'DOI', 'Link', 'Affiliations', 'Authors with affiliations', 'Abstract',
       'Author Keywords', 'Index Keywords', 'Document Type',
       'Publication Stage', 'Access Type', 'Source', 'EID'],
      dtype='object')

In [48]:
columns = ['Authors', 'Author(s) ID', 'Title', 'Year', 'Source title', 'Volume',
       'DOI', 'Affiliations','Index Keywords', 'Document Type']

In [49]:
data = data[columns]
data.head()

,Authors,Author(s) ID,Title,Year,Source title,Volume,DOI,Affiliations,Index Keywords,Document Type
0,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,2021,Advances in Intelligent Systems and Computing,1210 AISC,10.1007/978-3-030-51758-8_15,"Computer Science Department, Brigham Young Uni...",Drones; Human engineering; Intelligent systems...,Conference Paper
1,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,2021,Lecture Notes in Electrical Engineering,695 LNEE,10.1007/978-3-030-58653-9_3,"INESC-TEC Technology and Science, Campus da FE...",Automation; Cascade control systems; Computati...,Conference Paper
2,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,Development of sediment management guidelines ...,2020,Journal of Hydraulic Engineering,146,10.1061/(ASCE)HY.1943-7900.0001822,"Task Committee Secretary and Lead Engineer, Op...",Hydraulics; Editorial board; Sediment manageme...,Review
3,"Bunnell S., Gorrell S., Salmon J., Thelin C., ...",57204031131;6602649851;56830076500;57204029674...,Structural design space exploration using prin...,2020,Journal of Computing and Information Science i...,20,10.1115/1.4047428,"Department of Mechanical Engineering, Brigham ...",Compressors; Data handling; Structural design;...,Article
4,"Richardson M., Cannon S., Teichert L., Vance A...",57217149830;57217149403;57217145377;5721714653...,Religion-focused dating apps: A Q methodology ...,2020,Telematics and Informatics,55,10.1016/j.tele.2020.101448,"School of Communications, Brigham Young Univer...",Entertainment; A-RINGS; External pressures; Ro...,Article


In [50]:
# remove same articles
title_set = set()
unique_index_title = []
for i,t in enumerate(data['Title']):
    if t not in title_set:
        title_set.add(t)
        unique_index_title.append(i)

In [51]:
data = data.loc[unique_index_title]

In [52]:
data['Title'].value_counts()

Self-assessment of Proficiency of Intelligent Systems: Challenges and Opportunities    1
Universal airfoil parametrization using B-splines                                      1
Over-design versus redesign as a response to future requirements                       1
Design and modeling of a prosthetic venous valve                                       1
Patterned carbon nanotube growth on stainless steel                                    1
                                                                                      ..
The personification of big data                                                        1
Cooperating in long-term relationships with time-varying structure                     1
Modeling pressurized dense phase coal fluidization and transport                       1
A usability study of five two-factor authentication methods                            1
Dryland agriculture in north america                                                   1
Name: Title, Length: 

## Modifying Documents Type

In [53]:
data['Document Type'].value_counts()

Article             436
Conference Paper    392
Review               23
Book Chapter         16
Editorial             6
Note                  3
Erratum               2
Book                  1
Short Survey          1
Name: Document Type, dtype: int64

In [54]:
data['Article Type'] = 'Conference Paper'

In [55]:
journal_indexs = list(data[data['Source title'].str.contains(pat='Journal')].index)

In [56]:
data['Article Type'][journal_indexs] = "Journal"

/tmp/ipykernel_462986/2631974531.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Article Type'][journal_indexs] = "Journal"


In [57]:
data['Article Type'].value_counts()

Conference Paper    710
Journal             170
Name: Article Type, dtype: int64

In [58]:
data.describe()

,Year
count,880.000000
mean,2018.354545
std,1.027152
min,2017.000000
25%,2017.000000
50%,2018.000000
75%,2019.000000
max,2021.000000


In [59]:
len(data)

880

In [60]:
data.fillna('', inplace=True)

## Inserting article IDs

In [61]:
idx = 0
art_ids = list(range(1,len(data)+1))  # can be a list, a Series, an array or a scalar   
data.insert(loc=idx, column='Article ID', value=art_ids)

## Seperating Nodes

In [76]:
# Authors
Authors_Dict= {}
# To get unique affiliations
author_set = set()


# unique conferences/journal 
Publications ={}


# unique affilations
authors_organizations = {}


# unique Keywords
keywords_dict = {}

In [77]:
data.head()

,Article ID,Authors,Author(s) ID,Title,Year,Source title,Volume,DOI,Affiliations,Index Keywords,Document Type,Article Type
0,1,"Gautam A., Crandall J.W., Goodrich M.A.",57218202833;7004904337;7005513246;,Self-assessment of Proficiency of Intelligent ...,2021,Advances in Intelligent Systems and Computing,1210 AISC,10.1007/978-3-030-51758-8_15,"Computer Science Department, Brigham Young Uni...",Drones; Human engineering; Intelligent systems...,Conference Paper,Conference Paper
1,2,"de Moura Oliveira P.B., Hedengren J.D., Boaven...",6508306234;9277159100;6507358470;,Bridging theory to practice: Feedforward and c...,2021,Lecture Notes in Electrical Engineering,695 LNEE,10.1007/978-3-030-58653-9_3,"INESC-TEC Technology and Science, Campus da FE...",Automation; Cascade control systems; Computati...,Conference Paper,Conference Paper
2,3,"Hajimirzaie S.M., Hotchkiss R.H.",53879700900;26642910500;,Development of sediment management guidelines ...,2020,Journal of Hydraulic Engineering,146,10.1061/(ASCE)HY.1943-7900.0001822,"Task Committee Secretary and Lead Engineer, Op...",Hydraulics; Editorial board; Sediment manageme...,Review,Journal
3,4,"Bunnell S., Gorrell S., Salmon J., Thelin C., ...",57204031131;6602649851;56830076500;57204029674...,Structural design space exploration using prin...,2020,Journal of Computing and Information Science i...,20,10.1115/1.4047428,"Department of Mechanical Engineering, Brigham ...",Compressors; Data handling; Structural design;...,Article,Journal
4,5,"Richardson M., Cannon S., Teichert L., Vance A...",57217149830;57217149403;57217145377;5721714653...,Religion-focused dating apps: A Q methodology ...,2020,Telematics and Informatics,55,10.1016/j.tele.2020.101448,"School of Communications, Brigham Young Univer...",Entertainment; A-RINGS; External pressures; Ro...,Article,Conference Paper


In [78]:
publication_num = 1
keyword_num = 1
organizations_num = 1


for i, row in data.iterrows():
    
    # extracting keyowrds
    keywords = row['Index Keywords'].split(";")
    for l,key_wrd in enumerate(keywords):
        key_wrd = key_wrd.strip()
        if key_wrd!='' and key_wrd not in keywords_dict:
            keywords_dict[key_wrd] = keyword_num
            keyword_num+=1
    
    
    
    # extracting authors
    authors = row['Authors'].split(",")
    author_Ids = row['Author(s) ID'].split(";")
    for k,ids in enumerate(author_Ids):
        if ids!='' and ids not in Authors_Dict:
            Authors_Dict[ids] = authors[k].strip()
    
    
    
    # extract affiliations
    aff =row['Affiliations'].split(";")
    for j,org in enumerate(aff):
        if org not in authors_organizations:
            authors_organizations[org] = organizations_num
            organizations_num+=1
            
            
    # extract conferences        
    conf = row['Source title']
    if conf not in Publications:
        Publications[conf] = [publication_num,row['Article Type']]
        publication_num+=1

In [79]:
AUTHOR_IDS = list(Authors_Dict)

In [80]:
keywords_dict

{'Drones': 1,
 'Human engineering': 2,
 'Intelligent systems': 3,
 'Real time systems': 4,
 'Security of data': 5,
 'Autonomous systems': 6,
 'Causal relationships': 7,
 'Decision mechanism': 8,
 'Dependency graphs': 9,
 'Real-time application': 10,
 'Self assessment': 11,
 'Social robots': 12,
 'Automation': 13,
 'Cascade control systems': 14,
 'Computation theory': 15,
 'Process control': 16,
 'Soft computing': 17,
 'Students': 18,
 'Cascade control': 19,
 'Cascade control structure': 20,
 'Control concept': 21,
 'Control techniques': 22,
 'Engineering course': 23,
 'Proportional , integral and derivative controllers': 24,
 'Teaching/learning': 25,
 'Theory and practice': 26,
 'Control theory': 27,
 'Hydraulics': 28,
 'Editorial board': 29,
 'Sediment management': 30,
 'Reservoir management': 31,
 'guideline': 32,
 'reservoir': 33,
 'sediment': 34,
 'Compressors': 35,
 'Data handling': 36,
 'Structural design': 37,
 'Compressor blades': 38,
 'Computational costs': 39,
 'Design space 

In [81]:
Authors_Dict

{'57218202833': 'Gautam A.',
 '7004904337': 'Crandall J.W.',
 '7005513246': 'Goodrich M.A.',
 '6508306234': 'de Moura Oliveira P.B.',
 '9277159100': 'Hedengren J.D.',
 '6507358470': 'Boaventura-Cunha J.',
 '53879700900': 'Hajimirzaie S.M.',
 '26642910500': 'Hotchkiss R.H.',
 '57204031131': 'Bunnell S.',
 '6602649851': 'Gorrell S.',
 '56830076500': 'Salmon J.',
 '57204029674': 'Thelin C.',
 '57204032253': 'Ruoti C.',
 '57217149830': 'Richardson M.',
 '57217149403': 'Cannon S.',
 '57217145377': 'Teichert L.',
 '57217146534': 'Vance A.',
 '57217147399': 'Kramer I.',
 '57217147006': 'Barter M.',
 '57210571665': 'King J.',
 '55611356500': 'Callahan C.',
 '57218886193': 'Stanković A.M.',
 '57210260262': 'Sarić A.A.',
 '6701777223': 'Sarić A.T.',
 '8448666000': 'Transtrum M.K.',
 '36149254500': 'Hackney D.',
 '57191155872': 'Goode T.',
 '56681156600': 'Seng F.',
 '35088650000': 'Pankow M.',
 '7201741468': 'Schultz S.',
 '7402786794': 'Peters K.',
 '57191822951': 'Coburn J.',
 '57188654880': '

In [82]:
Publications

{'Advances in Intelligent Systems and Computing': [1, 'Conference Paper'],
 'Lecture Notes in Electrical Engineering': [2, 'Conference Paper'],
 'Journal of Hydraulic Engineering': [3, 'Journal'],
 'Journal of Computing and Information Science in Engineering': [4, 'Journal'],
 'Telematics and Informatics': [5, 'Conference Paper'],
 'Electric Power Systems Research': [6, 'Conference Paper'],
 'Optical Fiber Technology': [7, 'Conference Paper'],
 'Computers and Graphics (Pergamon)': [8, 'Conference Paper'],
 'International Journal of Electrical Power and Energy Systems': [9,
  'Journal'],
 'Nanotechnology': [10, 'Conference Paper'],
 'Journal of Cleaner Production': [11, 'Journal'],
 'Journal of Manufacturing Processes': [12, 'Journal'],
 'IEEE Robotics and Automation Letters': [13, 'Conference Paper'],
 'Computer Methods in Applied Mechanics and Engineering': [14,
  'Conference Paper'],
 'IEEE Vehicular Technology Magazine': [15, 'Conference Paper'],
 'Journal of Nondestructive Evaluati

In [83]:
authors_organizations

{'Computer Science Department, Brigham Young University, Provo, UT  84604, United States': 1,
 'INESC-TEC Technology and Science, Campus da FEUP, Porto, Portugal': 2,
 ' Department of Engineering, University of Trás-os-Montes and Alto Douro (UTAD), Vila Real, Portugal': 3,
 ' Department of Chemical Engineering, Brigham Young University, Provo, UT, United States': 4,
 'Task Committee Secretary and Lead Engineer, Operational Hydraulics Unit, Hydrology and Hydraulics (HandH) Bureau, South Florida Water Management District, 3301 Gun Club Rd., West Palm Beach, FL  33406, United States': 5,
 ' Dept. of Civil and Environmental Engineering, Brigham Young Univ., 430 Engineering Bldg., Provo, UT  84602, United States': 6,
 'Department of Mechanical Engineering, Brigham Young University, Provo, UT  8460, United States': 7,
 ' Pratt and Whitney, East Hartford, CT  06118, United States': 8,
 'School of Communications, Brigham Young University, United States': 9,
 ' Department of Communication, Univ

In [84]:
# articles and relations
Article_dict=[]


#--------------------- Relations
# author id, school id
author_belongs_org = []


# auth id, article id
author_wrote_art=[]


# paper id, cited id (paper only)
paper_cited = []


# article id, publication id
art_published_in = []


# article id, authorid
art_reviewed_by = []


# article id, keywords
art_key_rel = []

In [85]:
def author_school_preprocess(articleid,authors,author_Ids,author_affs):
    no_of_auth = len(authors)
    if len(author_affs) < no_of_auth:
        author_affs = author_affs*no_of_auth
    for i in range(no_of_auth):
        
        # add author article relationship
        author_wrote_art.append([author_Ids[i],articleid])
        
        if author_Ids[i] not in author_set:
            author_set.add(author_Ids[i])
            
            #affilation relation
            org_name = author_affs[i]
            author_belongs_org.append([author_Ids[i],authors_organizations[org_name]])      

In [86]:
for index, row in data.iterrows():
    authors = row['Authors'].split(",")
    author_Ids = row['Author(s) ID'].split(";")
    author_affs = row['Affiliations'].split(";")
    article_year = int(row['Year'])
    
    
    # authors and schools
    author_school_preprocess(row['Article ID'],authors,author_Ids,author_affs)
    
    
    
    # Articles
    Article_dict.append([row['Article ID'],row['Title'],row['Volume'],row['DOI']])
    
    
    
    #article keywords
    keywords = row['Index Keywords'].split(";")
    for l,key_wrd in enumerate(keywords):
        key_wrd = key_wrd.strip()
        if key_wrd!='':
            art_key_rel.append([row['Article ID'],keywords_dict[key_wrd]])
    
    
    
    # Citations
    no_of_citation = random.randint(1,7)
    for i in range(no_of_citation):
        cited_paper_id = random.randint(1,len(data))
        citation_year = random.randint(article_year-2,article_year)
        paper_cited.append([row['Article ID'],cited_paper_id,citation_year])
    
    
    # article published_in
    conf_id = Publications[row['Source title']][0]
    art_published_in.append([row['Article ID'],conf_id,row['Year']])
    
    
    # article reviewed by
    for i in range(3):
        random_reviewer_no = random.randint(1, len(AUTHOR_IDS)-1)
        reviewer_id = AUTHOR_IDS[random_reviewer_no]
        art_reviewed_by.append([row['Article ID'],reviewer_id,"This is an interesting study and the authors have collected a unique dataset using cuttingedge methodology. The paper is generally well written and structured","accapted"])

# Save files

In [87]:
# # Authors
# Authors_Dict= {}
# # To get unique affiliations
# author_set = set()


# # unique conferences/journal 
# Publications ={}


# # unique affilations
# authors_organizations = {}


# # unique Keywords
# keywords_dict = {}

In [91]:
Publications

{'Advances in Intelligent Systems and Computing': [1, 'Conference Paper'],
 'Lecture Notes in Electrical Engineering': [2, 'Conference Paper'],
 'Journal of Hydraulic Engineering': [3, 'Journal'],
 'Journal of Computing and Information Science in Engineering': [4, 'Journal'],
 'Telematics and Informatics': [5, 'Conference Paper'],
 'Electric Power Systems Research': [6, 'Conference Paper'],
 'Optical Fiber Technology': [7, 'Conference Paper'],
 'Computers and Graphics (Pergamon)': [8, 'Conference Paper'],
 'International Journal of Electrical Power and Energy Systems': [9,
  'Journal'],
 'Nanotechnology': [10, 'Conference Paper'],
 'Journal of Cleaner Production': [11, 'Journal'],
 'Journal of Manufacturing Processes': [12, 'Journal'],
 'IEEE Robotics and Automation Letters': [13, 'Conference Paper'],
 'Computer Methods in Applied Mechanics and Engineering': [14,
  'Conference Paper'],
 'IEEE Vehicular Technology Magazine': [15, 'Conference Paper'],
 'Journal of Nondestructive Evaluati

In [88]:
# authors
columns_auth = ["ID","name"]
authors = []
for i,j in Authors_Dict.items():
    authors.append([i,j])
auth = pd.DataFrame(authors,columns = columns_auth)
auth.to_csv('kaggle_data_processed/authors.csv',index=False)

In [92]:
#conferences
columns_conf = ["ID","name","Type"]
conferences = []
for i,j in Publications.items():
    conferences.append([j[0],i,j[1]])
conferences = pd.DataFrame(conferences,columns = columns_conf)
conferences.to_csv('kaggle_data_processed/publications.csv',index=False)

In [93]:
# authors_schools
columns_auth = ["ID","name"]
schools = []
for i,j in authors_organizations.items():
    schools.append([j,i])
scl = pd.DataFrame(schools,columns = columns_auth)
scl.to_csv('kaggle_data_processed/schools.csv',index=False)

In [94]:
# keywords
columns_auth = ["ID","name"]
keys = []
for i,j in keywords_dict.items():
    keys.append([j,i])
ky_word = pd.DataFrame(keys,columns = columns_auth)
ky_word.to_csv('kaggle_data_processed/concepts.csv',index=False)

In [95]:
# # articles and relations
# Article_dict=[]


# #--------------------- Relations
# # author id, school id
# author_belongs_org = []

# # auth id, article id
# author_wrote_art=[]

# # paper id, cited id (paper only)
# paper_cited = []

# # article id, publication id
# art_published_in = []

# # article id, authorid
# art_reviewed_by = []


# # article id, keywords
# art_key_rel = []

In [100]:
author_wrote_art

[['57218202833', 1],
 ['7004904337', 1],
 ['7005513246', 1],
 ['6508306234', 2],
 ['9277159100', 2],
 ['6507358470', 2],
 ['53879700900', 3],
 ['26642910500', 3],
 ['57204031131', 4],
 ['6602649851', 4],
 ['56830076500', 4],
 ['57204029674', 4],
 ['57204032253', 4],
 ['57217149830', 5],
 ['57217149403', 5],
 ['57217145377', 5],
 ['57217146534', 5],
 ['57217147399', 5],
 ['57217147006', 5],
 ['57210571665', 5],
 ['55611356500', 5],
 ['57218886193', 6],
 ['57210260262', 6],
 ['6701777223', 6],
 ['8448666000', 6],
 ['36149254500', 7],
 ['57191155872', 7],
 ['56681156600', 7],
 ['35088650000', 7],
 ['7201741468', 7],
 ['7402786794', 7],
 ['57191822951', 8],
 ['56830076500', 8],
 ['57188654880', 8],
 ['57192247939', 9],
 ['57218886193', 9],
 ['6701777223', 9],
 ['8448666000', 9],
 ['57218593962', 10],
 ['57218593446', 10],
 ['57218591162', 10],
 ['57201772566', 10],
 ['16422844400', 10],
 ['7102975249', 10],
 ['57193338296', 11],
 ['57194692502', 11],
 ['57202017958', 11],
 ['57205128747', 

In [96]:
# articles
columns_art = ["ID","title","volume","DOI"]
articles = pd.DataFrame(Article_dict,columns = columns_art)
articles.to_csv('kaggle_data_processed/articles.csv',index=False)

In [97]:
# author_belongs_org
columns_art = ["author_ID","org_ID"]
author_belongs_school = pd.DataFrame(author_belongs_org,columns = columns_art)
author_belongs_school.to_csv('kaggle_data_processed/author_belongs_school.csv',index=False)

In [101]:
# author_wrote_art
columns_art = ["author_ID","article_ID"]
author_written_article = pd.DataFrame(author_wrote_art,columns = columns_art)
author_written_article.to_csv('kaggle_data_processed/author_written_article.csv',index=False)

In [102]:
# paper_cited
columns_art = ["article_ID","cite_paper_ID","year"]
paper_cited_by = pd.DataFrame(paper_cited,columns = columns_art)
paper_cited_by.to_csv('kaggle_data_processed/paper_cited_by.csv',index=False)

In [103]:
# art_published_in
columns_art = ["article_ID","publisher_ID","year"]
article_published_by = pd.DataFrame(paper_cited,columns = columns_art)
article_published_by.to_csv('kaggle_data_processed/article_published_by.csv',index=False)

In [104]:
# art_reviewed_by
columns_art = ["article_ID","author_ID","description","decision"]
article_reviewed_by = pd.DataFrame(art_reviewed_by,columns = columns_art)
article_reviewed_by.to_csv('kaggle_data_processed/article_reviewed_by.csv',index=False)

In [105]:
# art_key_rel
columns_art = ["article_ID","keyword_ID"]
article_has_keyword = pd.DataFrame(art_key_rel,columns = columns_art)
article_has_keyword.to_csv('kaggle_data_processed/article_has_keyword.csv',index=False)